# CSV Analysis: Country

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 400 # Avoid "..." in large strings

## Loading CSV data from Clea

We will analyze the content of
[this CSV file](https://drive.google.com/file/d/1XmBh6YlfPkB5WfYSolAMP1EA5e02jHQO/view?usp=sharing)
created [here](https://github.com/scieloorg/normalizations-experiments/blob/master/experiments_2018-06-04.ipynb)
with [SciELO's Clea](https://github.com/scieloorg/clea)
from a selection of SciELO's articles corpora (XML documents).

In [2]:
dataset = pd.read_csv("inner_join_2018-06-04.csv",
                      dtype=str,
                      keep_default_na=False) \
            .drop_duplicates()

In [3]:
dataset.describe()[[k for k in dataset.columns if k.startswith("addr")]]

,addr_city,addr_country,addr_country_code,addr_postal_code,addr_state
count,89451,89451,89451,89451,89451
unique,2729,267,125,426,672
top,,Brazil,BR,,
freq,9529,48429,56596,87918,20906


The address information is incomplete in most entries:
87918 documents don't have a postal code.
But for the remaining columns
it seems that most rows have some information regarding the address.
Data is unbalanced and has unmatched fields:

- Brazil appears far more often than other countries;
- Brazil and BR don't count the same.

There are $266$ distinct country names and $124$ distinct country codes in this dataset
($+1$ of each if we count the empty entries):

In [4]:
print(", ".join(dataset["addr_country"].unique()))

Brazil, Portugal, Brasil, Uruguay, Spain, Italy, Argentina, Venezuela, Germany, USA, United Kingdom, República Argentina, France, England, Itália, França, Dinamarca, Colombia, Switzerland, Canada, Pakistan, Cuba, México, Mexico, Costa Rica, China, Saudi Arabia, BraZil, Greece, Espanha, , Alemanha, Estados Unidos, Australia, Malaysia, Jamaica, India, United States, Unites States, Paraguay, España, Chile, Sweden, Panama, UAE, Egypt, Qatar, Nigeria, Sri Lanka, Belgium, Denmark, U.K, Turkey, Iran, Russia, Italia, Poland, Dominican Republic, Índia, Austrália, Tailândia, People's Republic of China, Tunisia, United Arab Emirates, Azerbaijan, Romania, Bulgaria, Thailand, Japan, PR China, Republic of Korea, Slovakia, Taiwan, Algeria, Korea, Kingdom of Bahrain, Austria, BR, Netherlands, South Korea, Bangladesh, Serbia, Hong Kong, South Africa, Peru, Canadá, Colômbia, Equador, Slovenia, China., The Netherlands, Brazil., UK, Bolivia, BO, Ireland, sofialopezmdp@gmail.com, EUA, Guiana, Portuga, Braz

In [5]:
len(dataset["addr_country"].unique())

267

In [6]:
dataset["addr_country_code"].unique()

array(['BR', 'PT', '', 'UY', 'ES', 'IT', 'AR', 'VE', 'DE', 'US', 'GB',
       'FR', 'DK', 'CO', 'CH', 'CA', 'PK', 'CU', 'MX', 'CR', 'CN', 'SA',
       'GR', 'AT', 'MY', 'JM', 'IN', 'SE', 'AU', 'CL', 'PA', 'AE', 'EG',
       'QA', 'NG', 'LK', 'BE', 'TR', 'IR', 'RU', 'PL', 'TN', 'RO', 'BG',
       'TH', 'JP', 'KP', 'SK', 'TW', 'DZ', 'KR', 'NL', 'BD', 'RS', 'HK',
       'ZA', 'PE', 'EC', 'SI', 'BO', 'IE', 'GY', 'BR; BR', 'CY', 'HU',
       'IQ', 'PY', 'UK', 'HR', 'UA', 'FI', 'IL', 'MD', 'NI', 'NO', 'JO',
       'NZ', 'GD', 'SW', 'BV', 'AG', 'UG', 'CZ', 'CS', 'HN', 'BJ', 'MG',
       'ME', 'TA', 'TZ', 'ZM', 'MZ', 'AO', 'LU', 'PR', 'GH', 'ID', 'BF',
       'PF', 'BM', 'MK', 'EE', 'VN', 'PO', 'SN', 'MU', 'CM', 'MA', 'BA',
       'TK', 'SZ', 'GE', 'OM', 'AL', 'IS', 'LY', 'SD', 'LB', 'KZ', 'SV',
       'GT', 'DO', 'SB', 'LT', 'BY'], dtype=object)

In [7]:
len(dataset["addr_country_code"].unique())

125

But more than 20 thousand rows don't include the country code.
That's quite a lot! Also, this data is clearly unbalanced:

In [8]:
pd.DataFrame(dataset.groupby("addr_country_code")
                    .size()
                    .sort_values(ascending=False)
                    .head(10),
             columns=["count"])

,count
addr_country_code,
BR,56596
,21727
CN,1421
PT,1021
US,989
TR,851
AR,690
ES,599
CO,540


As we've already seen, the top `addr_country_code` ($56596$ `BR` rows)
and top `addr_country` ($48429$ `Brazil` rows) should have had the same value,
but there's a difference,
the country names and codes aren't always filled together,
and there are spurious country names filled in some rows:

In [9]:
dataset.groupby(["addr_country", "addr_country_code"]).size().sort_values(ascending=False).head(30)

addr_country   addr_country_code
Brazil         BR                   38521
Brasil         BR                   17187
Brazil                               9895
Brasil                               5883
                                     2829
China          CN                    1296
Portugal       PT                    1007
Turkey         TR                     844
Argentina      AR                     683
               BR                     638
USA            US                     627
Colombia       CO                     507
Spain          ES                     425
Portugal                              420
Iran           IR                     413
India          IN                     398
Chile          CL                     369
China                                 363
Turkey                                334
Mexico         MX                     310
USA                                   237
México         MX                     223
France         FR                     221
I

There's even some country codes filled as country names:

In [10]:
import re
from unidecode import unidecode

In [11]:
def pre_normalize(name):
    return " ".join(re.sub("[^a-z ]", "", unidecode(name).lower()).split())

In [12]:
data_countries = dataset["addr_country"].apply(pre_normalize)
pd.DataFrame({"count": data_countries[data_countries.apply(len) == 2].value_counts()})

,count
br,154
uk,81
sp,10
am,2
ru,2
be,2
rs,1
us,1
al,1
fr,1


There's no contributor referencing multiple countries at once in this dataset,
there's just one document referencing Brazil twice:

In [13]:
dataset[dataset["addr_country_code"].str.contains(";")]["addr_country_code"].unique()

array(['BR; BR'], dtype=object)

In [14]:
dataset[dataset["addr_country_code"] == "BR; BR"].T

,6353
addr_city,Goiás
addr_country,Brazil; Brazil
addr_country_code,BR; BR
addr_postal_code,
addr_state,GO
aff_email,
aff_id,aff1
aff_text,Federal University at Goiás (UFG) Universidade Federal de Goiás Federal University at Goiás Brazil Goiás GO Brazil
article_doi,10.1590/1982-02592018v21n1p09
article_publisher_id,


Our goal is to find what can be said about the country data of this dataset.

## Graph approach

Looking only to the country name (`addr_country` column)
and to the country code (`addr_country_code` column),
let's see what we can find.
Some possible questions are:

- Is the country pair inconsistent?
- Which inconsistencies are more common?

We won't be able to fill the correct data without some extra information
about the document being analyzed,
unless we use some *prior*,
which would just be a bias towards the unbalanced data we have
(e.g. both `Brasil, CL` and `Chile, BR` would yield `Brasil, BR`,
 because that's the most common pair).
On the other hand,
this *more common* approach for a single country name value
out of its row context
is meaningful:
the name `Chile` is assigned to `CL`,
as `CL` is the code that most `Chile` entries have.
What we can't say is whether the name or the code is wrong
in a given inconsistent row.

So let's find if there are inconsistencies, and whose are they.

In [15]:
pairs = dataset.groupby(["addr_country", "addr_country_code"]).size()
non_empty = pd.DataFrame(
    pairs[(pairs.index.get_level_values("addr_country").str.strip() != "") &
          (pairs.index.get_level_values("addr_country_code").str.strip() != "")],
    columns=["count"]
)
non_empty.head(14)

count
addr_country        addr_country_code       
AL                  AL                     1
Alemanha            DE                    27
Algeria             DZ                    31
Algerie             DZ                     3
Algérie             DZ                     2
Angola              AO                     2
                    AU                     1
                    BR                     1
Antigua and Barbuda AG                     1
Argentin            AR                     2
Argentina           AR                   683
                    BR                    10
Australia           AT                     6
                    AU                    83

These are the top entries, not the worst conflicts,
but there are clear conflicts and multiple languages in there.
Some highlights:

- **AL** is *Albania*, but is that correct in that single entry?
Sometimes we can't trust in neither column values;

- *Algérie* in French, *Argéria* in Portuguese and *Algeria* in English
are a single country with multiple names
due to multiple document languages in this dataset.
At least the country code **DZ** is correct;

- In the ISO 3166-1 alpha-2 code,
*Angola* is **AO**,
*Argentina* is **AR**,
*Australia* is **AU**,
*Austria* is **AT** and
*Brazil* is **BR**.
But there are clearly mixed rows, and they're not just a few
($10$ out of $693$ *Argentina* entries have **BR** as their code).

We can select some country codes to see what's going on:

In [16]:
codes = ["BR", "BV", "CH", "CN", "ES", "FR", "MX", "PT", "TR", "US"]

The country names connected to these codes,
segregated by the number of entries (*count*), is:

In [17]:
df_pairs = pairs.reset_index(name="count")
df_pairs_code = df_pairs[df_pairs["addr_country_code"].isin(codes)]

In [18]:
pd.DataFrame(df_pairs_code.groupby(["addr_country_code", "count"])
                          .apply(lambda grp: sorted(grp["addr_country"])),
             columns=["List of country names"])

List of country names
addr_country_code count                                                                                                                                                           
BR                1      [Angola, Braszil, Brazl, China, Colômbia, FR, Germany, India, Italy, Japan, KSA, Mexico, Moçambique, Nigeria, PA, RS, Slovenia, São Paulo, Venezuela, br]
                  2                                                                                                                       [Brasil., França, Not Normalized, Spain]
                  3                                                                                         [BRASIL, BRAZIL, Chile, Malaysia, People's Republic of China, Uruguay]
                  4                                                                                                                           [Brasill, Portugal, SP, Turkey, USA]
                  5                                                                                                                                      [BraZil, Espanha, México]
                  8                                                                                                                                                          [Bra]
                  10                                                                                                                                                   [Argentina]
                  16                                                                                                                                                     [Brazil.]
                  56                                                                                                                                                          [Br]
                  79                                                                                                                                                          [BR]
                  638                                                                                                                                                           []
                  17187                                                                                                                                                   [Brasil]
                  38521                                                                                                                                                   [Brazil]
BV                1                                                                                                                                                       [Brasil]
                  4                                                                                                                                                       [Brazil]
CH                1                                                                                                                                                        [China]
                  8                                                                                                                                                        [Chile]
                  10                                                                                                                                                       [Suíça]
                  29                                                                                                                                                 [Switzerland]
CN                1                                                                                                                                        [Henan, Taiwan, Urumqi]
                  2                                                                                                                                           [Canada, P. R.China]
                  6                     

The country name has several valid versions we can understand,
but there's too much information in that table:
we don't need to see the names that are always connected to a single code.

The following names are connected to the formerly selected country codes,
but they aren't connected to a single country code:

In [19]:
addr_country_counts = df_pairs_code["addr_country"].value_counts()
mixed_names = addr_country_counts[addr_country_counts > 1].index.tolist()
mixed_names

['',
 'Brazil',
 'Brasil',
 'USA',
 'China',
 'BR',
 'França',
 'Portugal',
 'Spain',
 'Turkey',
 'México',
 'Mexico',
 'Espanha',
 'Chile',
 "People's Republic of China"]

Let's see all the pairs regarding these names and the formerly selected codes:

In [20]:
df_pairs_code[df_pairs_code["addr_country"].isin(mixed_names)] \
     .sort_values(["addr_country_code", "addr_country"], ascending=True) \
     .set_index(["addr_country_code", "addr_country"])

count
addr_country_code addr_country                     
BR                                              638
                  BR                             79
                  Brasil                      17187
                  Brazil                      38521
                  Chile                           3
                  China                           1
                  Espanha                         5
                  França                          2
                  Mexico                          1
                  México                          5
                  People's Republic of China      3
                  Portugal                        4
                  Spain                           2
                  Turkey                          4
                  USA                             4
BV                Brasil                          1
                  Brazil                          4
CH                Chile                           8
                  China                           1
CN                China                        1296
                  People's Republic of China     25
ES                                                2
                  Brasil                          1
                  Brazil                          1
                  Espanha                        45
                  Spain                         425
                  USA                             2
FR                BR                              1
                  França                         53
MX                Mexico                        310
                  México                        223
PT                                                1
                  Brasil                          4
                  Brazil                          6
                  Portugal                     1007
TR                                                1
                  Brazil                          1
                  Turkey                        844
US                                                4
                  Brasil                          1
                  USA                           627

There are many unfilled and inconsistent pairs in this subset of our data.

Nevertheless, the whole data denotes a bipartite graph,
where the two partitions are the *names* and the *codes*,
and each $(name, code)$ pair is an edge.

### Adjacency matrix correlation

This bipartite graph can be depicted as an adjacency matrix.

In [21]:
adj_mat = non_empty.unstack(fill_value=0).xs("count", axis=1, drop_level=True)
adj_mat.shape

(241, 124)

A part of it would be:

In [22]:
adj_mat.iloc[:12, :9]

addr_country_code,AL,DE,DZ,AO,AU,BR,AG,AR,AT
addr_country,,,,,,,,,
AL,1,0,0,0,0,0,0,0,0
Alemanha,0,27,0,0,0,0,0,0,0
Algeria,0,0,31,0,0,0,0,0,0
Algerie,0,0,3,0,0,0,0,0,0
Algérie,0,0,2,0,0,0,0,0,0
Angola,0,0,0,2,1,1,0,0,0
Antigua and Barbuda,0,0,0,0,0,0,1,0,0
Argentin,0,0,0,0,0,0,0,2,0
Argentina,0,0,0,0,0,10,0,683,0


That clearly shows that (not exhaustive):

- There are $27$ connections between the country name *Alemanha* and the code **DE**;
- There are at least $3$ distinct country names connected to the **DZ** code;
- There are at least $3$ distinct codes connected to the name *Angola*;

But that's still a surface-level information.
It's not something easy to compare,
but we can grasp some information using the columns/rows correlations.

We can measure some similarity (correlation) between distinct columns:

In [23]:
br_adj_mat_corr = adj_mat.corrwith(adj_mat["BR"]).sort_values()
br_adj_mat_corr[abs(br_adj_mat_corr) > .01]

addr_country_code
BM    0.912946
BV    0.984703
BR    1.000000
dtype: float64

For connectedness alone
(i.e., using the *this pair either have or doesn't have a connection* without any weight),
we would have:

In [24]:
bin_adj_mat = (adj_mat != 0).applymap(int)
br_bin_adj_mat_corr = bin_adj_mat.corrwith(bin_adj_mat["BR"]).sort_values()
len(br_bin_adj_mat_corr[abs(br_bin_adj_mat_corr) > .01])

123

As only one entry is below that absolute correlation value threshold,
it shouldn't be so small in this connectedness-only scenario. 

In [25]:
br_bin_adj_mat_corr[abs(br_bin_adj_mat_corr) > .15]

addr_country_code
PT    0.154367
MX    0.190914
BV    0.190914
ES    0.265135
BR    1.000000
dtype: float64

Wait there!
Why such a high correlation before and a low correlation afterwards?!
What does that correlation mean anyway?

In [26]:
br_corr_similar_codes = ["PT", "MX", "BV", "ES", "BM", "BR"]
br_like = adj_mat[br_corr_similar_codes]
br_like[br_like.T.apply(sum) > 0].sort_values(by=br_corr_similar_codes)

addr_country_code,PT,MX,BV,ES,BM,BR
addr_country,,,,,,
Angola,0,0,0,0,0,1
Braszil,0,0,0,0,0,1
Brazl,0,0,0,0,0,1
China,0,0,0,0,0,1
Colômbia,0,0,0,0,0,1
FR,0,0,0,0,0,1
Germany,0,0,0,0,0,1
India,0,0,0,0,0,1
Italy,0,0,0,0,0,1


These last 3 rows shows the link between Portugal and Brasil:
there are *Brazil*-**PT**, *Brasil*-**PT** and *Portugal*-**BR** entries,
which makes **BR** and **PT** connectedness similar,
but not enough for a high correlation
due to the several "alternative names" (including typos)
connected to **BR** but not to **PT**,
besides the other two *Portugal* names (the uppercase and `Portugual`).
As it seems, we should not ignore the weights,
at least not the weights relative to the row/column in analysis.

The codes **BM** and **BV** have no other country but Brazil,
though **BM** stands for *Bermuda*
and **BV** stands for *Bouvet Island*,
which makes their entries look like typos:

In [27]:
br_like[br_like[["BM", "BV"]].T.apply(sum) > 0][["BM", "BV", "BR"]]

addr_country_code,BM,BV,BR
addr_country,,,
Brasil,0,1,17187
Brazil,1,4,38521


The same analysis can be performed on the rows
to find the distinct *Brazil* names/aliases/typos:

In [28]:
brazil_adj_mat_corr = adj_mat.T.corrwith(adj_mat.T["Brazil"]).sort_values()
brazil_adj_mat_corr[abs(brazil_adj_mat_corr) > .1]

addr_country
Espanha                       0.102466
People's Republic of China    0.111177
Moçambique                    0.116081
Angola                        0.401014
BR                            0.999920
BRASIL                        1.000000
BRAZIL                        1.000000
br                            1.000000
KSA                           1.000000
Not Normalized                1.000000
Bra                           1.000000
FR                            1.000000
PA                            1.000000
Brasil.                       1.000000
São Paulo                     1.000000
Brasill                       1.000000
Braszil                       1.000000
Brazil.                       1.000000
Brazl                         1.000000
RS                            1.000000
SP                            1.000000
Br                            1.000000
BraZil                        1.000000
Brasil                        1.000000
Brazil                        1.000000
dtype: float

Why *Spain*, *China*, *Mozambique* and *Angola* (the first four entries) are there?
Why the *BR* name is less related to *Brazil* than *Br*?

In [29]:
brazil_similar_condition = (abs(brazil_adj_mat_corr) > .1) & (abs(brazil_adj_mat_corr) < 1 - 1e-7)
brazil_corr_similar_codes = list(brazil_adj_mat_corr[brazil_similar_condition].index)
brazil_like = adj_mat.T[brazil_corr_similar_codes]
brazil_like[brazil_like.T.apply(sum) > 0].T

addr_country_code,AO,AU,BR,FR,ES,CN,MZ
addr_country,,,,,,,
Espanha,0,0,5,0,45,0,0
People's Republic of China,0,0,3,0,0,25,0
Moçambique,0,0,1,0,0,0,8
Angola,2,1,1,0,0,0,0
BR,0,0,79,1,0,0,0


It's still difficult to tell why the *Not Normalized* rows and the *KSA*
have such a high correlation to *Brazil*,
but what can be said about the data above is:

- The *BR* name has an entry connected to the **FR** code.
That's perhaps
  due to the *French Guiana*
    (**GF** is the country code, but the dataset doesn't include it),
  due to the *Brittany* metropolitan region
    (**FR-BRE** is the *ISO-3166-2* code), or
  due to a single character typo;

- *Angola* (**AO**) has only $2$ entries,
the other $2$ have distinct codes (**AU** and **BR**),
so the statistics are high due to the lack of actual information from such country;

- *Spain* (**ES**) when written as *Espanha* (i.e., in Portuguese)
have **BR** as its country code in $10.00\%$ of its entries,
perhaps
  due to the document language, or
  due to a non-intentional replacement of an *ES* input
    regarding to *Espírito Santo*
    (Brazilian state that has *ES* as its code)
    to *Espanha*;

- *China* (**CN**) when written as *People's Republic of China* (i.e., in English)
have **BR** as its country code in $10.71\%$ of its entries,
perhaps due to an incomplete copy-and-paste-and-fix approach when preparing the article;

- *RS* (*Rio Grande do Sul*), *SP* (*São Paulo*) and *PA* (*Pará*) are Brazilian states,
some entries have their code/names filled in as the country name.

Summary is, the not-so-small correlation between *Brazil* and *Angola*
(as well as the ones with *Spain*, *China* and *Mozambique* instead of *Angola*)
aren't as small as they could have been
because there are just a few entries for these countries.

### Lack of a pre-normalization

Perhaps we should remove the accents
and put everything in a common case
before using the graph.
Each name set like `BR`/`Br` and `Brazil`/`BraZil`/`BRAZIL` would become
a single row in the adjacency matrix by summing its components rows.
For some countries that would give us a single name as result:

In [30]:
mexico_adj_mat_corr = adj_mat.T.corrwith(adj_mat.T["Mexico"]).sort_values()
mexico_adj_mat_corr[abs(mexico_adj_mat_corr) > .02]

addr_country
México    0.999557
Mexico    1.000000
dtype: float64

But these are already very similar in every case,
is that required at all?

This graph approach to find similar names is completely different from
the DBSCAN name clustering solution based on Levenshtein distances
seen on [2018-05-03](experiments_2018-05-03.ipynb),
since every distinct country name is known *a priori*
(i.e., we know which names should belong to distinct classes),
and the distinct names might have nothing to do with typos,
but with a whole another way to write the same country name
(e.g. due to it being written in another language or due to the extra words,
like *People's Republic of China* instead of just *China*).

For the graph, the pre-normalization might not be required at all,
but we can perform everything the other way around:
use the correlation between the two names as the "distance"
in the DBSCAN algorithm to create country name clusters.